In [357]:
from datetime import date, timedelta

from Stock import *

s = date(2020,1,1)
e = date(2021,12,20)
tesla = Stock("tsla")
tesla.load_data()
#tesla.add_data_range(s,e,stockpath='pricedata/tsla.csv')
#tesla.save_data()

# adding individual days
#tesla.add_data(s)

done loading $tsla data.


In [400]:
len(tesla.df)

6701

In [878]:
# Analysis
import seaborn as sns
import matplotlib.pyplot as plt
import re

df = tesla.df.dropna()
df['domain'] = df.apply(
    lambda row: re.sub('www3.|www.','',row.link.split('//')[1].split('/')[0]), axis=1)
df1 = df.iloc[0:len(df)//2] #df.sample(frac=0.5)
df2 = df.drop(df1.index)
print(len(df))
print(len(df1))
print(len(df2))

4048
2024
2024


/tmp/ipykernel_3769/2263100439.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['domain'] = df.apply(


In [879]:
count = {}
for p, tx in df1[['percent_change','text']].values:
    tx = re.sub('[^a-zA-Z ]',' ',tx)
    for word in tx.split():
        if word.lower() not in count:
            count[word.lower()] = [0,0]
        count[word.lower()][p < 0] += 1

In [880]:
prob = {k:list(np.array(count[k])/(sum(count[k]))) for k in count}

In [881]:
len(prob)

21549

In [882]:
print('--------- positive ---------')
for k in sorted(prob,key=prob.get,reverse=True):
    if prob[k][0] >= 0.70 and sum(count[k]) > 25:
        print(k,prob[k],count[k])
print('--------- negative ---------')
for k in sorted(prob,key=prob.get,reverse=False):
    if prob[k][1] >= 0.70 and sum(count[k]) > 25:
        print(k,prob[k],count[k])

--------- positive ---------
xi [1.0, 0.0] [26, 0]
balan [1.0, 0.0] [26, 0]
redwood [0.9615384615384616, 0.038461538461538464] [25, 1]
squeeze [0.9024390243902439, 0.0975609756097561] [37, 4]
beijing [0.8793103448275862, 0.1206896551724138] [51, 7]
walmart [0.8666666666666667, 0.13333333333333333] [26, 4]
comparing [0.8620689655172413, 0.13793103448275862] [25, 4]
nvidia [0.8571428571428571, 0.14285714285714285] [36, 6]
gabriel [0.8461538461538461, 0.15384615384615385] [22, 4]
petition [0.82, 0.18] [41, 9]
nations [0.8148148148148148, 0.18518518518518517] [22, 5]
sparks [0.8148148148148148, 0.18518518518518517] [22, 5]
twenty [0.8064516129032258, 0.1935483870967742] [25, 6]
patents [0.7954545454545454, 0.20454545454545456] [35, 9]
plane [0.7931034482758621, 0.20689655172413793] [23, 6]
forty [0.7878787878787878, 0.21212121212121213] [26, 7]
unintended [0.7878787878787878, 0.21212121212121213] [26, 7]
iron [0.7878787878787878, 0.21212121212121213] [26, 7]
trim [0.7857142857142857, 0.214

In [883]:
rel = {}
for k in sorted(prob,key=prob.get,reverse=True):
    if prob[k][0] >= 0.60 and sum(count[k]) > 75:
        rel[k] = prob[k]
for k in sorted(prob,key=prob.get,reverse=False):
    if prob[k][1] >= 0.60 and sum(count[k]) > 25:
        rel[k] = prob[k]

In [884]:
len(rel)

921

In [323]:
accuracy = 0
for num in range(len(df2)):
    prediction = [0,0]
    count = [0, 0]
    tx = df2['text'].iloc[num]
    tx = re.sub('[^a-zA-Z ]',' ',tx)
    for word in tx.split():
        if word.lower() in prob:
            prediction[prob[word.lower()][0] < 0.5] += prob[word.lower()][prob[word.lower()][0] < 0.5]
            count[prob[word.lower()][0] < 0.5] += 1
    if count[0] > 0:
        prediction[0] /= count[0]
    if count[1] > 0:
        prediction[1] /= count[1]
    prediction[0] /= sum(prediction)
    prediction[1] =  1 - prediction[0]
    #print(prediction,df2['percent_change'].iloc[num])
    accuracy += int((df2['percent_change'].iloc[num] < 0) == (prediction[0] < 0))
accuracy /= len(df2)
print(accuracy)

/tmp/ipykernel_3769/2690515489.py:9: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  prediction[prob[word.lower()][0] < 0.5] += prob[word.lower()][prob[word.lower()][0] < 0.5]
/tmp/ipykernel_3769/2690515489.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  count[prob[word.lower()][0] < 0.5] += 1


0.5138339920948617


In [885]:
prob['germany']

[0.48148148148148145, 0.5185185185185185]

In [886]:
def encode_text(tx):
    enc = np.zeros(len(rel))
    tx = re.sub('[^a-zA-Z ]',' ',tx)
    tx = tx.split()
    #for i, k in enumerate(prob):
    #    if k in tx:
    #        enc[i] = 1
    enc = [int(k in tx) for k in rel]
    return enc

In [887]:
dataset = df2.apply(
    lambda row: [row.percent_change,encode_text(row.text)],
    axis=1,result_type='expand')
dataset.columns = ['label','data']

In [888]:
labels = dataset['label'].values.astype(np.float64)
inputs = np.array(dataset['data'].values.tolist())

In [889]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.3, shuffle=True)

In [890]:
#from sklearn.ensemble import AdaBoostRegressor
#clf = AdaBoostRegressor(loss='square')
#clf.fit(X_train, y_train)
#print(clf.score(X_test,y_test))

from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(X_train, (y_train < 0).astype(int))
print(clf.score(X_test,(y_test < 0).astype(int)))

0.4769736842105263


In [891]:
#from sklearn.ensemble import RandomForestRegressor
#clf = RandomForestRegressor()
#clf.fit(X_train, y_train)
#print(clf.score(X_test,y_test))

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, (y_train < 0).astype(int))
print(clf.score(X_test,(y_test < 0).astype(int)))

0.5592105263157895


In [1153]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(max_iter=1000, tol=1e-10,shuffle=True)
clf.fit(X_train, (y_train < 0).astype(int))
print(clf.score(X_train,(y_train < 0).astype(int)))

0.8891242937853108


In [1154]:
print(clf.score(X_test,(y_test < 0).astype(int)))

0.5592105263157895


In [978]:
#https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/

In [1165]:
time_res = np.array([int(r == (y_test[i] < 0)) for i,r in enumerate(clf.predict(X_test))])

In [1166]:
# average length of correct guesses in a row
length = 0
count = 0
avg = 0
for x in time_res:
    if x == 0:
        count+=1
        avg += length
        length = 0
    elif x == 1:
        length+=1
avg /= count
avg

1.2649253731343284